# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [54]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [55]:
df = pd.read_csv("Dataframes/characteristics.csv")
X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,1.532200e+09,4.413001e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
19335,POOL.OQ,2021-10-01,Retailing,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
19336,POOL.OQ,2022-01-01,Retailing,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
19337,POOL.OQ,2022-04-01,Retailing,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
19338,POOL.OQ,2022-07-01,Retailing,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1


In [56]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [57]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_panel = df_clean.copy()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,1.532200e+09,4.413001e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
19335,POOL.OQ,2021-10-01,Retailing,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
19336,POOL.OQ,2022-01-01,Retailing,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
19337,POOL.OQ,2022-04-01,Retailing,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
19338,POOL.OQ,2022-07-01,Retailing,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1


#### Random Effects

In [58]:
# turn dataframe into right dimension for panel regression with multiindex
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)

# define the dependent variable



# perform the random effects panel regression
random_effects_model = RandomEffects(y, X)

# fit the model and print the summary statistics
random_effects_results = random_effects_model.fit()
print(random_effects_results.summary)


                                 RandomEffects Estimation Summary                                 
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                        0.0030
Estimator:                                RandomEffects   R-squared (Between):             -0.3708
No. Observations:                                 18569   R-squared (Within):               0.0112
Date:                                  Sun, Feb 26 2023   R-squared (Overall):             -0.0274
Time:                                          21:44:22   Log-likelihood                -7.829e+04
Cov. Estimator:                              Unadjusted                                           
                                                          F-statistic:                      9.1615
Entities:                                           501   P-value                           0.0000
Avg Obs:                                         37.064   Distribution:                 F(6,18562)
Min Obs:  

**R-squared (Within)**: This is the proportion of variation in the dependent variable (in your case, Earnings Per Share - Actual Surprise) that is explained by the independent variables (in our case, the variables in X) within each individual instrument (the 501 companies). In other words, this R-squared measures how well the independent variables explain the variation in the dependent variable (surprise) within each instrument. The R-squared (Within) is 0.0112, which means that the independent variables explain only about 1.12% of the variation in the dependent variable within each instrument.  

**R-squared (Between)**: This is the proportion of variation in the dependent variable that is explained by the independent variables across the instruments. In other words, this R-squared measures how well the independent variables explain the variation in the dependent variable between different instruments. The R-squared (Between) is -0.3708, which means that the independent variables actually perform worse than just using the mean of the dependent variable for each instrument to explain the variation in the dependent variable across entities.  

**R-squared (Overall)**: This is the proportion of variation in the dependent variable that is explained by the independent variables overall, taking into account both the within-instrument and between-instrument variation. The R-squared (Overall) is -0.0274, which means that the independent variables explain only about 2.74% of the variation in the dependent variable overall.

#### Fixed Effects

In [59]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean[X_var_names]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


Dep. Variable:,Earnings Per Share - Actual Surprise,R-squared:,-0.0029
Estimator:,PanelOLS,R-squared (Between):,-0.8660
No. Observations:,18569,R-squared (Within):,0.0056
Date:,"Sun, Feb 26 2023",R-squared (Overall):,-0.0932
Time:,21:44:22,Log-likelihood,-7.773e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,-8.6700
Entities:,501,P-value,1.0000
Avg Obs:,37.064,Distribution:,"F(6,18022)"
Min Obs:,6.0000,,
Max Obs:,40.000,F-statistic (robust):,482.92


By Industry: example tech industry

In [62]:
df2 = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']
groups = df2.groupby('Instrument')

def panel_regression_random(group):
    if len(group) < 2:
        return None
    
    group['Instrument'] = group['Instrument'].astype('category')
    group['Date'] = pd.to_datetime(group['Date'])
    group.set_index(['Instrument', 'Date'], inplace=True)

    y = group.loc[:, "Earnings Per Share - Actual Surprise"]
    X = group[X_var_names]

    model = RandomEffects(y, X).fit()
    return model


results_random = groups.apply(panel_regression_random)
results_random = results_random.dropna()
models_random = results_random.tolist()
models_random

[                                 RandomEffects Estimation Summary                                 
 Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                        0.4285
 Estimator:                                RandomEffects   R-squared (Between):              0.0000
 No. Observations:                                    40   R-squared (Within):               0.4285
 Date:                                  Sun, Feb 26 2023   R-squared (Overall):              0.4285
 Time:                                          21:45:10   Log-likelihood                   -132.40
 Cov. Estimator:                              Unadjusted                                           
                                                           F-statistic:                      4.1245
 Entities:                                             1   P-value                           0.0034
 Avg Obs:                                         40.000   Distribution:                    F(6,33)
